In [1]:
import itertools
import warnings

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.stats as st
import scipy.special

# The MCMC Hammer
import emcee

import numba

# BE/Bi 103 utilities
import bebi103

# Import plotting tools
import matplotlib.pyplot as plt
import seaborn as sns
import corner

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables high res graphics inline (only use with static plots (non-Bokeh))
# SVG is preferred, but there is a bug in Jupyter with vertical lines
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_csv("./data/gardner_hw6/gardner_mt_catastrophe_only_tubulin.csv", comment = "#")

In [3]:
df.head()

,12 uM,7 uM,9 uM,10 uM,14 uM
0,25.000,35,25,50,60
1,40.000,45,40,60,75
2,40.000,50,40,60,75
3,45.429,50,45,75,85
4,50.000,55,50,75,115


In [20]:
tau = np.array([15,10,17])
t=100

#m=0
for i in range(2):
    if not tau[i]<tau[i+1]:
        sum_z = -np.inf
        break
        
    else:
        x=[] #d
        y=[] #num
        z=[] #function output
        x_array_product=[] #den

        for j in range(3):
            y.append( (tau[j]*np.exp(-t/tau[j]) ))

            for k in range(3):
                if k != j:
                    x.append(tau[j] - tau[k])
            x_array=np.array(x)
            x_array_product.append(np.prod(x_array))

            z.append(y[j]/x_array_product[j])
            x=[]
            #m+=1
        z_array=np.array(z)
        if np.sum(z)>0:
            sum_z = np.log(np.sum(z))
        else:
            RaiseError("WTF man!?")
    break
            

In [57]:
def log_likelihood(tau, time, m):
    """
    Takes in tau (should be an array), 
    data, various values for m. We are very proud of this function.
    """
    tau = np.sort(tau)
    
    num_handler = [] #numerator
    den_handler = [] #denominator
    den_elements = [] #elements necessary to take products in den
    func_handler = [] #output log posterior
    
    for i in range(m-1):

        if not tau[i]<tau[i+1]:

            return -np.inf
            
        for j in range(m):
            num_handler.append(tau[j]**(m-2) * np.exp(-time/tau[j]))

            for k in range(m):
                if k != j:
                    den_elements.append(tau[j] - tau[k])

            #den_elements_array = np.array(den_elements)

            den_handler.append(np.prod(den_elements))

            func_handler.append(num_handler[j]/den_handler[j])

            den_elements = []

        #func_handler_array = np.array(func_handler)

        if np.sum(func_handler)>0:
            return np.log(np.sum(func_handler))

        else:
            RaiseError("WTF Man.")

def log_prior(tau, m):
    """
    Log prior for model defined above. Takes in m and tau, 
    returns log prior.
    """
    tau = np.sort(tau)
    
    for i in range(m-1):

        if not tau[i]<tau[i+1]:

            return -np.inf
    
    if not np.all(tau> 0):
        return -np.inf
    
    return -np.log(np.prod(tau))
        
test_run = log_likelihood(tau, t, 3)

def tau_start(m, n_walkers, n_temps):
    p = np.empty((n_temps, n_walkers, m))
    
    for i in range(m):
        p[:,:,i]=np.random.exponential(0.1, (n_temps,n_walkers))
    return p


In [59]:
df["12 uM"].head()

0    25.000
1    40.000
2    40.000
3    45.429
4    50.000
Name: 12 uM, dtype: float64

In [56]:
n_burn=100
n_steps=1000
n_temps=20
p0=tau_start(3,50,20)
loglargs = (df["12 uM"], 3)
logpargs = (3,)
columns = 

array([[[  2.84690086e-01,   1.68073325e-02,   4.06292718e-02],
        [  5.33791043e-03,   1.25977050e-01,   3.87506352e-01],
        [  1.32440541e-02,   6.64423842e-02,   3.58689335e-01],
        [  1.16472755e-01,   6.66558534e-02,   2.08422572e-02],
        [  1.18888917e-02,   1.91170412e-02,   5.90565622e-03],
        [  7.74030947e-02,   2.51351044e-01,   2.80007336e-02],
        [  1.92030726e-01,   9.59816776e-02,   5.24402936e-02],
        [  1.75324931e-01,   7.74048892e-02,   7.40586603e-02],
        [  9.55384685e-02,   1.17473158e-01,   2.36379126e-01],
        [  3.62132509e-02,   1.71765589e-01,   2.41947459e-01]],

       [[  5.14147612e-03,   8.52354715e-02,   1.20027534e-01],
        [  3.63533569e-02,   3.39876845e-02,   1.50698303e-03],
        [  4.17073903e-02,   1.15943326e-01,   5.38388871e-03],
        [  2.50867882e-04,   3.30802930e-01,   2.45009984e-02],
        [  2.20849789e-01,   1.31438575e-02,   3.53019236e-02],
        [  2.79977348e-01,   3.2657849

In [49]:
np.sort(a)

array([-2,  1,  3])

In [ ]:
x=1
print(-x)

In [ ]:
x=[1,2,3]
np.prod(x)